In [1]:
import numpy as np

import time

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

from sklearn.datasets import load_boston

In [2]:
data, target = load_boston(return_X_y=True)
X, X_test, y, y_test = train_test_split(data, target)

X = np.insert(X, 0, 1, axis=1)
X_test = np.insert(X_test, 0, 1, axis=1)

In [3]:
def matrix_mul(matries):
    length = len(matries)
    if length < 3:
        raise Exception('Not enough matries.')
    
    first_matrix = matries.pop(0)
    I = np.identity(first_matrix.shape[0])
    mul = I.dot(first_matrix)
    while matries != []:
        next_matrix = matries.pop(0)
        mul = mul.dot(next_matrix)
    return mul

def print_r2_scores(names, betas):
    print('r2_scores: ')
    for name, beta in zip(names, betas):
        y_pred = X_test.dot(beta)
        score = r2_score(y_test, y_pred)
        print(name, ':', score)

In [4]:
# 1. LinearRegression

linear_regression = LinearRegression()
linear_regression.fit(X, y)
beta_linear = linear_regression.coef_

In [5]:
# 2. Iteratively reweighted least squares

def IRLS(X, y, p):
    n, m = X.shape
    
    W = np.zeros((n, n))
    w = np.ones(n)
    delta = 0.00001
    it = 1000
    
    beta = np.zeros(m)
    
    start_seconds = time.time()
    
    while it > 0:
        np.fill_diagonal(W, w)
    
        G = matrix_mul([X.T, W.T, W, X])
        G_inv = np.linalg.inv(G)

        beta_new = matrix_mul([G_inv, X.T, W.T, y])
        
        if np.allclose(beta, beta_new):
            print('it', it)
            beta = beta_new
            break
        
        r_abs = np.abs(y - X.dot(beta_new))
        r = np.where(r_abs > delta, r_abs, delta)
        
        w = np.power(r, (p-2)/2)
        beta = beta_new
        
        it -= 1
        
    seconds = time.time() - start_seconds
    print('seconds', seconds)
    return beta

In [6]:
beta_irls_1 = IRLS(X, y, 1)
beta_irls_1_5 = IRLS(X, y, 1.5)
beta_irls_2 = IRLS(X, y, 2)

it 913
seconds 0.1650080680847168
seconds 1.4859411716461182
it 999
seconds 0.004476070404052734


In [7]:
# 3. GIRLS

# h(r) = np.power((r*r+delta), 1/2)
# g(r, z) = h(r) + (z*z-r*r)/(2h(r))

def GIRLS(X, y, p):

    n, m = X.shape
    delta = 0.00001
    
    
    beta = np.ones(m)
    
    it = 1000
    W = np.zeros((n, n))
    
    start_seconds = time.time()

    while it > 0:
        r = y - X.dot(beta)
        w = np.power(r*r+delta, (p-2)/2)
        np.fill_diagonal(W, w)

        G = matrix_mul([X.T, W, X])
        G_inv = np.linalg.inv(G)

        beta_new = matrix_mul([G_inv, X.T, W, y])
        if np.allclose(beta, beta_new):
            beta = beta_new
            print('it', it)
            break
        else:
            beta = beta_new
            it -= 1
    seconds = time.time() - start_seconds
    print('seconds', seconds)
    return beta

In [8]:
beta_girls_1 = GIRLS(X, y, 1)
beta_girls_1_5 = GIRLS(X, y, 1.5)
beta_girls_2 = GIRLS(X, y, 2)

it 717
seconds 0.28119921684265137
it 982
seconds 0.018699169158935547
it 999
seconds 0.002808809280395508


In [9]:
# compare r2_scores

names = ['linear', 'irls_1', 'irls_1.5', 'irls_2', 'girls_1', 'girls_1.5', 'girls_2']
betas = [beta_linear, beta_irls_1, beta_irls_1_5, beta_irls_2,  beta_girls_1, beta_girls_1_5, beta_girls_2] 

print_r2_scores(names, betas)

r2_scores: 
linear : -12.644672428278124
irls_1 : -3715.708114980665
irls_1.5 : -13.707831873522439
irls_2 : 0.6271302394192783
girls_1 : 0.6067067809995353
girls_1.5 : 0.6225998071404956
girls_2 : 0.6271302394192783
